In [ ]:
#code to run a reciprocal blast on entire transcriptome (suberites domuncula vs mus musculus)

In [27]:
#python: 3.9.1
#not 3.9.6, 3.11.0., 3.10.0.,
import scanpy as sc
from Bio import SeqIO
import os
import pandas as pd
adata = sc.read_h5ad("/Users/work/Desktop/subdom.h5ad")
genes = list(adata.var.index)

proteome_file = "/Users/work/Desktop/mus_musculus/proteome/proteome_shorthand/mus_musculus.transcriptome.fasta"
transcriptome_file = "suberites_domuncula.longest_isoform.fasta"  # Path to your transcriptome fasta file

In [28]:
output_file = "tmp/sd_to_mm.fasta"  # Path to the output file to save the sequences

# Open the output file for writing
with open(output_file, "w") as output_handle:
    # Iterate over the selected genes in the specified order
    for selected_gene in genes:
        # Iterate over the sequences in the transcriptome fasta file
        for record in SeqIO.parse(transcriptome_file, "fasta"):
            gene_name = record.id  # Assuming the gene name is the sequence identifier
            # Check if the current gene matches the selected gene
            if gene_name == selected_gene:
                # Write the selected gene sequence to the output file
                SeqIO.write(record, output_handle, "fasta")
                break  # Exit the inner loop once the gene is found

#!./blast.sh tmp/myc_targets_sd.fasta tmp/1
!./blast.sh tmp/sd_to_mm.fasta tmp/1 && awk -F',' '!seen[$1]++' tmp/1 > tmp/2



Building a new DB, current time: 04/30/2024 09:20:17
New DB name:   /Users/work/Desktop/mus_musculus/proteome/proteome_shorthand/mus_musculus.transcriptome.fasta
New DB title:  /Users/work/Desktop/mus_musculus/proteome/proteome_shorthand/mus_musculus.transcriptome.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/work/Desktop/mus_musculus/proteome/proteome_shorthand/mus_musculus.transcriptome.fasta
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 54910 sequences in 1.30455 seconds.




In [29]:
file_path = "tmp/2" 

second_items = []

with open(file_path, 'r') as file:
    items=file.read().split("\n")
    for item in items[:len(items)-1]:
        second_items.append(str(item.split(",")[1]))

os.remove(output_file)
output_file = "tmp/mm_to_sd.fasta"

# Create a set to store the selected gene names
selected_genes_set = set(second_items)

# Open the output file for writing
with open(output_file, "w") as output_handle:
    # Iterate over the sequences in the proteome fasta file
    for record in SeqIO.parse(proteome_file, "fasta"):
        gene_name = record.id  # Assuming the gene name is the sequence identifier
        
        # Check if the current gene is in the set of selected genes
        if gene_name in selected_genes_set:
            # Write the selected gene sequence to the output file
            SeqIO.write(record, output_handle, "fasta")
            
            # Remove the gene from the selected_genes_set
            selected_genes_set.remove(gene_name)
            
            # Check if all selected genes have been found
            if len(selected_genes_set) == 0:
                break

!./recip_blast.sh tmp/mm_to_sd.fasta tmp/3 && awk -F',' '!seen[$1]++' tmp/3 > tmp/4



Building a new DB, current time: 04/30/2024 13:31:06
New DB name:   /Users/work/Library/Mobile Documents/com~apple~CloudDocs/Desktop/ADesktop/Studium/PhD/code/reciprocal_blast/suberites_domuncula.longest_isoform.fasta
New DB title:  suberites_domuncula.longest_isoform.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 14398 sequences in 0.487354 seconds.


BLAST Database error: No alias or index file found for nucleotide database [/Users/work/Library/Mobile] in search path [/Users/work/Library/Mobile Documents/com~apple~CloudDocs/Desktop/ADesktop/Studium/PhD/code/reciprocal_blast::]


In [30]:
input_file1 = "tmp/2"  # Path to the first input file
input_file2 = "tmp/4"  # Path to the second input file
full_blast_file = "tmp/full_transcriptome.xlsx"

with open(input_file1, 'r') as file:
    list1=file.read().split("\n")
with open(input_file2, 'r') as file:
    list2=file.read().split("\n")


# Split the elements of list1 into separate columns
list1_split = [item.split(',') for item in list1]
list1_df = pd.DataFrame(list1_split, columns=['Sponge gene', 'Mouse protein', 'e-value'])

# Split the elements of list2 into separate columns
list2_split = [item.split(',') for item in list2]
list2_df = pd.DataFrame(list2_split, columns=['Mouse protein', 'reciprocal sponge match', 'e-value'])

# Merge the two dataframes based on the 'Mouse ID' column
merged_df = pd.merge(list1_df, list2_df, on='Mouse protein', how='left')

# Save the merged dataframe to an Excel file
merged_df.to_excel(full_blast_file, index=False)

print("Merged data saved to '%s'." % full_blast_file)
os.remove(output_file)
os.remove(input_file1)
os.remove(input_file2)

/var/folders/tb/mtqjrwzn04v2ws67ymnhmv5h0000gp/T/ipykernel_44876/4247996276.py:23: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  merged_df.to_excel(full_blast_file, index=False)


Merged data saved to 'tmp/full_transcriptome.xlsx'.


In [32]:
file_path = full_blast_file

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path)

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [34]:
merged_df['Match'] = merged_df['Sponge gene'] == merged_df['reciprocal sponge match']


In [37]:
merged_df["Match"].value_counts()

Match
True     5211
False    3457
Name: count, dtype: int64